In [119]:
import aocd
import numpy as np
import heapq
from functools import cache
from itertools import pairwise, permutations

In [120]:
#useful from previous days...

In [121]:
test_codes = """
029A: <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
980A: <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A
179A: <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
456A: <v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A
379A: <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"""

In [122]:
def make_stringmap(arr):
    stringmap = []
    for line in arr:
        stringmap.append("".join(line))
    stringmap = "\n".join(stringmap)
    return stringmap

def turn_left(direction):
    # Define the 90-degree counterclockwise rotation matrix
    turn_left_matrix = np.array([[0, -1], [1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_left_matrix, direction))

def turn_right(direction):
    # Define the 90-degree clockwise rotation matrix
    turn_right_matrix = np.array([[0, 1], [-1, 0]])
    
    # Multiply the direction by the rotation matrix
    return tuple(np.dot(turn_right_matrix, direction))


In [123]:
numpad = """789
456
123
#0A"""

In [124]:
def make_array(data):
    return np.pad(np.array([[x for x in line]for line in data.split("\n")]),1, mode="constant", constant_values = "#")

In [125]:
numpad_arr = make_array(numpad)

In [126]:
numpad_arr

array([['#', '#', '#', '#', '#'],
       ['#', '7', '8', '9', '#'],
       ['#', '4', '5', '6', '#'],
       ['#', '1', '2', '3', '#'],
       ['#', '#', '0', 'A', '#'],
       ['#', '#', '#', '#', '#']], dtype='<U1')

In [127]:
dir_pad = """#^A
<v>"""

In [128]:
dirpad_arr = make_array(dir_pad)

In [129]:
dirpad_arr

array([['#', '#', '#', '#', '#'],
       ['#', '#', '^', 'A', '#'],
       ['#', '<', 'v', '>', '#'],
       ['#', '#', '#', '#', '#']], dtype='<U1')

In [130]:
inst_dict = {(-1,0):"^", (1,0):"v",(0,1):">", (0,-1):"<"}
rev_inst_dict = {inst_dict[k]: k for k in inst_dict}

In [131]:

#edit this to keep track of path and take arbirtary start and end chars for both numpads
@cache
def dijkstra(start_char = "S", end_char = "E"):
    if start_char in "<>^v" or end_char in "<^>v":
        arr = dirpad_arr
    else:
        arr = numpad_arr
    start = np.argwhere(arr == start_char)[0]
    visited = set()
    q = [(0, [start], [0,1])] #cost, pos, direction
    while q:
        cost, path, direction = heapq.heappop(q) #retrieve the lowest cost next step from the queue for checking
        loc = path[-1]
        if (tuple(loc), tuple(direction)) in visited: #if we have already been here, we are on the way back from a bad route and need to ignore the loc
            continue #we have done this
        if arr[loc[0], loc[1]] == end_char: #if we stumble upon the target tile, we have found the (or at least one) lowest cost route.
            #arrived, we return cost
            return "".join([inst_dict[tuple(x)] for x in np.diff(np.vstack(path), axis = 0)])
        visited.add((tuple(loc), tuple(direction))) #we record our current position and direction
        one_ahead = [loc[0] + direction[0], loc[1]+direction[1]] #look at the space one ahead of us:
        if arr[one_ahead[0], one_ahead[1]] != "#":
            heapq.heappush(q, (cost+1, path+[[one_ahead[0], one_ahead[1]]], direction)) #if the space one ahead of us is not a wall, it is an empty space or the end, and we need to explore it for a cost of 1
        #we also need to explore what happens if we turn left and right, for a cost of 1000 each.
        heapq.heappush(q, (cost+1, path, turn_left(direction)))
        heapq.heappush(q, (cost+1, path, turn_right(direction)))

In [132]:
dijkstra("A", ">")

'v'

In [133]:
test_code = "029A"
test_answer = "<A^A>^^AvvvA"
second_test_answer = "v<<A>>^A<A>AvA<^AA>A<vAAA>^A"
third_test_answer = "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A"

In [134]:
inst_dict = {(-1,0):"^", (1,0):"v",(0,1):">", (0,-1):"<"}
rev_inst_dict = {inst_dict[k]: k for k in inst_dict}

In [135]:
rev_inst_dict

{'^': (-1, 0), 'v': (1, 0), '>': (0, 1), '<': (0, -1)}

In [238]:
def is_valid(start, end, path, arr):
    tiles = []
    loc = start.copy()
    for p in path:
        d = rev_inst_dict[p]
        loc+=d
        if arr[loc[0], loc[1]] == "#":
            return False
    return True

In [257]:
#ok dijkstra is killing me, lets simplify this shit somehow
def get_simple_paths(start, end):
    if start in "<>^v" or end in "<^>v":
        arr = dirpad_arr
    else:
        arr = numpad_arr

    loc_start = np.argwhere(arr == start)[0]
    loc_end = np.argwhere(arr == end)[0]
    delta = loc_end-loc_start
    if delta[0] < 0:
        vertical = "^"
    else:
        vertical = "v"
    if delta[1] <0:
        horizontal = "<"
    else:
        horizontal = ">"
    path = vertical*abs(delta[0]) + horizontal*abs(delta[1])
    valid_paths = set()
    for p in set(permutations(path)):
        p = "".join(p)
        # print(loc_start, loc_end)
        if is_valid(loc_start, loc_end, p, arr):
            valid_paths.add(p+"A")
    return valid_paths
    
    

In [258]:
get_simple_paths("A", "7")

{'<^<^^A',
 '<^^<^A',
 '<^^^<A',
 '^<<^^A',
 '^<^<^A',
 '^<^^<A',
 '^^<<^A',
 '^^<^<A',
 '^^^<<A'}

In [259]:
@cache
def make_recursive_instruction(code, depth = 2):
    code = "A"+code
    # print(f"\nMaking instructions for {code}")
    len_path = 0
    for start, end in pairwise(code):
        all_paths = get_simple_paths(start, end)
        # print(f"testing {len(all_paths)} permutations of {first_path}")
        if depth==0:
            len_path += min([len(path) for path in all_paths])
        else:
            len_path += min([make_recursive_instruction(path, depth-1) for path in all_paths])
    return len_path
        

In [260]:
make_recursive_instruction("179A", depth = 2)

68

In [261]:
codes = """029A
980A
179A
456A
379A""".split("\n")

In [262]:
total_score = 0
for code in codes:
    complexity = make_recursive_instruction(code, depth=2)
    numpart = int(code.replace("A",""))
    total_score += (complexity*numpart)
    print(f"{code=}, {complexity} x {numpart}")

code='029A', 68 x 29
code='980A', 60 x 980
code='179A', 68 x 179
code='456A', 64 x 456
code='379A', 64 x 379


In [263]:
for a, b in pairwise("A"+code):
    print(a, b)

A 3
3 7
7 9
9 A


In [264]:
total_score

126384

In [265]:
make_recursive_instruction("179A", depth=2)

68

In [266]:
real_codes = aocd.get_data()

In [267]:
real_codes = real_codes.split("\n")

In [268]:
real_codes

['935A', '319A', '480A', '789A', '176A']

In [271]:
total_score = 0
for code in real_codes:
    complexity = make_recursive_instruction(code, depth=25)
    numpart = int(code.replace("A",""))
    total_score += (complexity*numpart)
    print(f"{code=}, {complexity} x {numpart}")

code='935A', 86475783010 x 935
code='319A', 82050061712 x 319
code='480A', 90594397580 x 480
code='789A', 80786362260 x 789
code='176A', 89741193602 x 176


In [272]:
total_score

230049027535970

In [114]:
def make_instructions(code:str, arr):
    code = "A"+code
    list_instructions = [[code[i], code[i+1]] for i in range(len(code)-1)]
    instructions = ""
    for inst in list_instructions:
        l, path = dijkstra(arr, inst[0], inst[1])
        for step in path:
            instructions+=inst_dict[tuple(step)]
        instructions+="A"
    return instructions

In [115]:
from itertools import permutations

In [316]:
dijkstra_all_paths(numpad_arr, "A", "5")

(5,
 [[array([4, 3]),
   [np.int64(3), np.int64(3)],
   [np.int64(2), np.int64(3)],
   [np.int64(2), np.int64(2)]]])

In [317]:
codes = """029A
980A
179A
456A
379A"""

In [318]:
make_instructions("379A", numpad_arr)

'^A^^<<A>>AvvvA'

In [319]:
codes = codes.split("\n")

In [320]:
codes

['029A', '980A', '179A', '456A', '379A']

In [321]:
def score_code(code):
    print("\n"+"#"*10)
    print(code)
    first_code = make_instructions(code, numpad_arr)
    print(first_code)
    second_code = make_instructions(first_code, dirpad_arr)
    print(second_code)
    third_code = make_instructions(second_code, dirpad_arr)
    print(third_code)
    numerical_part = int(code.rstrip("A"))
    print(f"{len(third_code)}, {numerical_part},  complexity = {numerical_part*len(third_code)}")
    return  numerical_part*len(third_code)

In [322]:
def solve1(codes):
    return sum([score_code(code) for code in codes])

In [323]:
solve1(codes)


##########
029A
<A^A>^^AvvvA
v<<A>>^A<A>AvA^<AA>Av<AAA>^A
v<A<AA>>^AvAA^<A>Av<<A>>^AvA^Av<A>^A<Av<A>>^AAvA^Av<A<A>>^AAAvA^<A>A
68, 29,  complexity = 1972

##########
980A
^^^A<AvvvA>A
<AAA>Av<<A>>^Av<AAA>^AvA^A
v<<A>>^AAAvA^Av<A<AA>>^AvAA^<A>Av<A<A>>^AAAvA^<A>Av<A>^A<A>A
60, 980,  complexity = 58800

##########
179A
^<<A^^A>>AvvvA
<Av<AA>>^A<AA>AvAA^Av<AAA>^A
v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AAvA^Av<A>^AA<A>Av<A<A>>^AAAvA^<A>A
68, 179,  complexity = 12172

##########
456A
^^<<A>A>AvvA
<AAv<AA>>^AvA^AvA^Av<AA>^A
v<<A>>^AAv<A<A>>^AAvAA^<A>Av<A>^A<A>Av<A>^A<A>Av<A<A>>^AAvA^<A>A
64, 456,  complexity = 29184

##########
379A
^A^^<<A>>AvvvA
<A>A<AAv<AA>>^AvAA^Av<AAA>^A
v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A>^AA<A>Av<A<A>>^AAAvA^<A>A
68, 379,  complexity = 25772


127900

In [324]:
real = "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
bad =  "v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A>^AA<A>Av<A<A>>^AAAvA^<A>A"

In [325]:
def simulate_presses(instructions, arr):
    loc = np.argwhere(arr == "A")[0]
    pressed = ""
    for instruction in instructions:
        if instruction == "A":
            pressed+=arr[loc[0], loc[1]]
        else:
            instruction = rev_inst_dict[instruction]
            loc = [loc[0] + instruction[0], loc[1] + instruction[1]]
    return pressed

In [326]:
def simulate_sequence(third_code):
    print(third_code)
    second_code=simulate_presses(third_code, dirpad_arr)
    to_print = ""
    for letter, seq in zip(second_code, third_code.split("A")):
        to_print+="."*len(seq)+letter
    print(to_print)
    first_code =simulate_presses(second_code, dirpad_arr)
    to_print2 = ""
    for letter, seq in zip(first_code, to_print.split("A")):
        to_print2+="."*len(seq)+letter
    print(to_print2)
    code = simulate_presses(first_code, numpad_arr)

    to_print3 = ""
    for letter, seq in zip(code, to_print2.split("A")):
        to_print3+="."*len(seq)+letter
    print(to_print3)
    return code

In [327]:
simulate_sequence(real)

<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
...<...A.>.A..v.<<...AA.>..^.AA.>.A..v..AA.^.A...<.v..AAA.>..^.A
.......^...A.........<<......^^...A.....>>...A........vvv......A
...........3......................7..........9.................A


'379A'

In [328]:
simulate_sequence(bad)

v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A>^AA<A>Av<A<A>>^AAAvA^<A>A
...<...A.>.A...<...AA..v.<...AA.>>..^.A..v..AA.^.A..v.<...AAA.>..^.A
.......^...A.......^^........<<.......A.....>>...A........vvv......A
...........3..........................7..........9.................A


'379A'

In [330]:
l, c = dijkstra(numpad_arr, "3", "7")

In [336]:
instructions = [inst_dict[tuple(i)] for i in c]

In [345]:
for x in permutations("".join(instructions)):
    print(simulate_sequence(make_instructions("".join(x), dirpad_arr)))

<AAv<AA
.^^..<<



<AAv<AA
.^^..<<



<Av<A>^Av<A
.^..<..^..<



<Av<AA>^A
.^..<<..^



<Av<A>^Av<A
.^..<..^..<



<Av<AA>^A
.^..<<..^



<AAv<AA
.^^..<<



<AAv<AA
.^^..<<



<Av<A>^Av<A
.^..<..^..<



<Av<AA>^A
.^..<<..^



<Av<A>^Av<A
.^..<..^..<



<Av<AA>^A
.^..<<..^



v<<A>^AAv<A
...<..^^..<



v<<A>^Av<A>^A
...<..^..<..^



v<<A>^AAv<A
...<..^^..<



v<<A>^Av<A>^A
...<..^..<..^



v<<AA>^AA
...<<..^^



v<<AA>^AA
...<<..^^



v<<A>^AAv<A
...<..^^..<



v<<A>^Av<A>^A
...<..^..<..^



v<<A>^AAv<A
...<..^^..<



v<<A>^Av<A>^A
...<..^..<..^



v<<AA>^AA
...<<..^^



v<<AA>^AA
...<<..^^



